In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
class Node:
    def __init__(self,predicted_class):
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None
class Decisiontree:
    def __init__(self,max_depth = None):
        self.max_depth = max_depth
        
    def fit(self, X, y):
        self.n_classes = len(set(y))
        self.n_features = X.shape[1]
        self.tree_ = self.tree(X, y)

    def find_split(self,X, y):
        """ Find split where children has lowest impurity possible
        in condition where the purity should also be less than the parent,
        if not, stop.
        """
        n_samples, n_features = X.shape
#         print(n_samples)
        if n_samples <= 1:
            return None, None

        #so it will not have any warning about "referenced before assignments"
        feature_ix, threshold = None, None

        # Count of each class in the current node.
        sample_per_class_parent = [np.sum(y == c) for c in range(self.n_classes)] #[2, 2]
    #     print(sample_per_class_parent)

        # Gini of parent node.
        best_gini = 1.0 - sum((n / n_samples) ** 2 for n in sample_per_class_parent)
    #     print(best_gini)

        # Loop through all features.
        for feature in range(self.n_features):

            # Sort data along selected feature.
            sample_sorted, y_sorted = zip(*sorted(zip(X[:, feature], y)))

            sample_per_class_left = [0] * self.n_classes   #[0, 0]

            sample_per_class_right = sample_per_class_parent.copy() #[2, 2]

            #loop through each threshold, 2.5, 6.5, 14.5
            #1st iter: [-] [-++]
            #2nd iter: [--] [++]
            #3rd iter: [--+] [+]
            for i in range(1, n_samples): #1 to 3 (excluding 4)
                #the class of that sample
                c = y_sorted[i - 1]  #[0]

                #put the sample to the left
                sample_per_class_left[c] += 1  #[1, 0]
    #             print("left",sample_per_class_left)

                #take the sample out from the right  [1, 2]
                sample_per_class_right[c] -= 1
    #             print("right",sample_per_class_right)
                gini_left = 1.0 - sum(
                    (sample_per_class_left[x] / i) ** 2 for x in range(self.n_classes)
                )

                #we divided by n_samples - i since we know that the left amount of samples
                #since left side has already i samples
                gini_right = 1.0 - sum(
                    (sample_per_class_right[x] / (n_samples - i)) ** 2 for x in range(self.n_classes)
                )

                #weighted gini
                weighted_gini = ((i / n_samples) * gini_left) + ( (n_samples - i) /n_samples) * gini_right

                # in case the value are the same, we do not split
                # (both have to end up on the same side of a split).
                if sample_sorted[i] == sample_sorted[i - 1]:
                    continue

                if weighted_gini < best_gini:
                    best_gini = weighted_gini
                    feature_ix = feature
                    threshold = (sample_sorted[i] + sample_sorted[i - 1]) / 2  # midpoint

        #return the feature number and threshold 
        #used to find best split
        return feature_ix, threshold
    def tree(self,Xtrain, ytrain, depth=0):  
        num_samples_per_class = [np.sum(ytrain == i) for i in range(self.n_classes)]
        predicted_class = np.argmax(num_samples_per_class)

        #define the parent node
        node = Node(predicted_class=predicted_class)

        #perform recursion
        if depth < self.max_depth:
            feature, threshold = self.find_split(Xtrain, ytrain)
#         print("Best feature used for split: ", feature)
#         print("Best threshold used for split: ", threshold)
            if feature is not None:
                #take all the indices that is less than threshold
                indices_left = X[:, feature] < threshold
    #             print(indices_left)
    #             print(~indices_left)
                X_left, y_left = X[indices_left], y[indices_left]
    #             print("X_left is",X_left)
    #             print("y_left is",y_left)

                #tilde for negation
                X_right, y_right = X[~indices_left], y[~indices_left]

                #take note for later decision
                node.feature_index = feature
                node.threshold = threshold
                node.left = self.tree(X_left, y_left, depth + 1)
                node.right = self.tree(X_right, y_right, depth + 1)
        return node
    def predict(self, X):
        return [self._predict(inputs) for inputs in X]
    def _predict(self, inputs):
        node = self.tree_
        while node.left:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class

In [3]:
import sys
from sklearn.datasets import load_iris

dataset = load_iris()
X, y = dataset.data, dataset.target
model = Decisiontree(max_depth=10)
model.fit(X, y)
print(model.predict([[0, 0, 5, 1.5]]))

[0]
